In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error

In [ ]:
df = pd.read_csv('data/football_matches_dataset_v2.csv')

In [ ]:
X = df[['home_team_avg_rating', 'away_team_avg_rating']]
y = df[['home_team_score', 'away_team_score']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **1.4 Model Training Stage**

In [ ]:
param_dist = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_features': ['auto', 'sqrt', 'log2'],
    'estimator__max_depth': [None, 10, 20, 30],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}
rf = RandomForestRegressor(random_state=42)
multi_output_rf = MultiOutputRegressor(rf)

# ========= randomizedSearchCV ============ #
random_search = RandomizedSearchCV(multi_output_rf, param_distributions=param_dist, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Best parameters found: ", best_params)
model = random_search.best_estimator_
y_pred= model.predict(X_test)
y_pred = np.round(y_pred)

mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)
rmse_best = np.sqrt(mse_best)

# ========== evaluation metrics ==============#
print("Mean Absolute Error:", mae_best)
print("Mean Squared Error:", mse_best)
print("Root Mean Squared Error:", rmse_best)

In [ ]:
joblib.dump(model, 'model/football_goals_prediction_model.pkl')